In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [21]:
train  = pd.read_csv('pid.dat', header = None)

yno = 7
yCol=train.columns[yno]
yCol

7

In [22]:
X = train.drop(yCol,1)
Y = train.iloc[:, yno]
df = pd.concat([X, Y], axis=1)

#tX = test.drop(yCol,1)
#tY = test.iloc[:, yCol]
#tdf = pd.concat([tX, tY], axis=1)

In [23]:
unqCnt = []
for i in range(X.shape[1]):
    unqCnt += [len(set(X.iloc[:,i]))]
uqCnt = np.array(unqCnt)

cateVarInd = np.where(uqCnt<=10)[0] # index of the categoral values
numerVarInd = np.where(uqCnt>10)[0] # index of the numerical values

In [25]:
def calcGini(vec):
    return 1 - (sum(vec==min(Y))/len(vec))**2-(sum(vec==max(Y))/len(vec))

In [26]:
def searchCoff(df , cInd):
    out = []
    for i in sorted(list(set(df[cInd])))[:-1]:
        g1 = df[df[cInd] <= i]
        g2 = df[df[cInd] > i]
        wGini = len(g1)*calcGini(g1[yCol])+len(g2)*calcGini(g2[yCol])
        out.append([wGini,i])
    return sorted(out , key = lambda x : x[0])[0]

In [27]:
ret = []
for i in numerVarInd:
    ret.append([i]+searchCoff(df,i))

In [161]:
ret

[[0, 44.50428396572826, 6],
 [1, 39.420875420875426, 146],
 [2, 49.708552317247964, 75],
 [3, 48.6525709584533, 20],
 [4, 48.152036199095015, 27.8],
 [5, 47.18294296163724, 0.626],
 [6, 45.49074074074073, 28]]

In [122]:
res = sorted(ret, key = lambda x : x[1])[0]

[1, 39.420875420875426, 146]

In [89]:
g1 = df[df[res[0]] <= res[2]]
g2 = df[df[res[0]] > res[2]]

In [96]:
if Counter(g1[yno])[min(Y)] > Counter(g1[yno])[min(Y)]:
    g1val = min(Y)
    g2val = max(Y)
else:
    g1val = max(Y)
    g2val = min(Y)

In [98]:
def getYhat(vec):
    if vec[res[0]] <= res[2]:
        return g1val
    else:
        return g2val

In [116]:
rslt= []
for i in range(X.shape[0]):
    rslt.append(getYhat(X.iloc[i,:]))
rslt = np.array(rslt)

cont = pd.crosstab(Y,rslt)
cont.columns.name = 'Resub Class'
cont.index.name = 'Actual Class'
acc = sum(np.diag(cont))/len(Y)
spe = cont.iloc[0,0]/sum(cont.iloc[0,:])
sens = cont.iloc[1,1]/sum(cont.iloc[1,:])
predy = pd.DataFrame([range(1,(len(X)+1)),Y,rslt]).transpose()
predy.columns=['ID','Actual class','Resub pred']

In [143]:
trslt= []
for i in range(tX.shape[0]):
    trslt.append(getYhat(tX.iloc[i,:]))
trslt = np.array(trslt)

tcont = pd.crosstab(tY,trslt)
tcont.columns.name = 'Test Class'
tcont.index.name = 'Actual Class'
tacc = sum(np.diag(tcont))/len(tY)
tspe = tcont.iloc[0,0]/sum(tcont.iloc[0,:])
tsens = tcont.iloc[1,1]/sum(tcont.iloc[1,:])
tpredy = pd.DataFrame([range(1,(len(tX)+1)),tY,trslt]).transpose()
tpredy.columns=['ID','Actual class','Test pred']

In [145]:
print(predy.to_string(index=False))
print(cont)
print("Overall accuracy = %0.3f" %(acc))
print("Sensitivity = %0.3f" %(sens))
print("Specificity = %0.3f" %(spe))

print(tpredy.to_string(index=False))
print(tcont)
print("Overall accuracy = %0.3f" %(tacc))
print("Sensitivity = %0.3f" %(tsens))
print("Specificity = %0.3f" %(tspe))

ID  Actual class  Resub pred
  1             2           2
  2             2           2
  3             2           2
  4             2           2
  5             2           2
  6             2           2
  7             2           2
  8             2           2
  9             2           2
 10             2           2
 11             2           2
 12             2           2
 13             2           2
 14             2           2
 15             2           2
 16             2           2
 17             2           2
 18             2           1
 19             2           2
 20             2           2
 21             2           2
 22             2           2
 23             2           1
 24             2           2
 25             2           2
 26             2           2
 27             2           2
 28             2           2
 29             2           2
 30             2           2
 31             2           2
 32             2           2
 33        

In [142]:
#file output
out = open(fname, 'w')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(predy.to_string(index=False),file=out)
    print("",file=out)
    print("Confusion Matrix (Resubstitution)",file=out)
    print(cont,file=out)
    print("",file=out)
    print("Model Summary (Resubstitution)",file=out)
    print("Overall accuracy = %0.3f" %(acc),file=out)
    print("Sensitivity = %0.3f" %(sens),file=out)
    print("Specificity = %0.3f" %(spe),file=out)
out.close()

252

In [150]:
def DT1lv():
    #input term
    trfname=input("Write your train data file name : ")
    tstfname=input("Write your test data file name : ")
    sep=input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")
    if sep=='1':
        form=' '
    elif sep=='2':
        form=','


    hd=input("Select the data header format(1 = with header or 2 = no header): ")
    if hd=='1':
        header='infer'
    elif hd=='2':
        header=None


    cl=input("Select column# where your y vlaues exist: ")
    yCol = int(cl)-1

    train = pd.read_csv(trfname,sep=form,header=header)
    test = pd.read_csv(tstfname,sep=form,header=header)
    X = train.drop(yCol,1)
    Y = train.iloc[:, yCol]
    df = pd.concat([X, Y], axis=1)

    tX = test.drop(yCol,1)
    tY = test.iloc[:, yCol]
    tdf = pd.concat([tX, tY], axis=1)
    unqCnt = []
    for i in range(X.shape[1]):
        unqCnt += [len(set(X.iloc[:,i]))]
    uqCnt = np.array(unqCnt)

    cateVarInd = np.where(uqCnt<=10)[0] # index of the categoral values
    numerVarInd = np.where(uqCnt>10)[0] # index of the numerical values

    def calcGini(vec):
        return 1 - (sum(vec==min(Y))/len(vec))**2-(sum(vec==max(Y))/len(vec))

    def searchCoff(df , cInd):
        out = []
        for i in sorted(list(set(df[cInd])))[:-1]:
            g1 = df[df[cInd] <= i]
            g2 = df[df[cInd] > i]
            wGini = len(g1)*calcGini(g1[yCol])+len(g2)*calcGini(g2[yCol])
            out.append([wGini,i])
        return sorted(out , key = lambda x : x[0])[0]

    ret = []
    for i in numerVarInd:
        ret.append([i]+searchCoff(df,i))

    res = sorted(ret, key = lambda x : x[1])[0]

    g1 = df[df[res[0]] <= res[2]]
    g2 = df[df[res[0]] > res[2]]

    if Counter(g1[7])[min(Y)] > Counter(g1[7])[min(Y)]:
        g1val = min(Y)
        g2val = max(Y)
    else:
        g1val = max(Y)
        g2val = min(Y)

    def getYhat(vec):
        if vec[res[0]] <= res[2]:
            return g1val
        else:
            return g2val

    rslt= []
    for i in range(X.shape[0]):
        rslt.append(getYhat(X.iloc[i,:]))
    rslt = np.array(rslt)

    cont = pd.crosstab(Y,rslt)
    cont.columns.name = 'Resub Class'
    cont.index.name = 'Actual Class'
    acc = sum(np.diag(cont))/len(Y)
    spe = cont.iloc[0,0]/sum(cont.iloc[0,:])
    sens = cont.iloc[1,1]/sum(cont.iloc[1,:])
    predy = pd.DataFrame([range(1,(len(X)+1)),Y,rslt]).transpose()
    predy.columns=['ID','Actual class','Resub pred']

    trslt= []
    for i in range(tX.shape[0]):
        trslt.append(getYhat(tX.iloc[i,:]))
    trslt = np.array(trslt)

    tcont = pd.crosstab(tY,trslt)
    tcont.columns.name = 'Test Class'
    tcont.index.name = 'Actual Class'
    tacc = sum(np.diag(tcont))/len(tY)
    tspe = tcont.iloc[0,0]/sum(tcont.iloc[0,:])
    tsens = tcont.iloc[1,1]/sum(tcont.iloc[1,:])
    tpredy = pd.DataFrame([range(1,(len(tX)+1)),tY,trslt]).transpose()
    tpredy.columns=['ID','Actual class','Test pred']

    fname = input('Write your output file name : ')
    
    print(predy.to_string(index=False))
    print(cont)
    print("Overall accuracy = %0.3f" %(acc))
    print("Sensitivity = %0.3f" %(sens))
    print("Specificity = %0.3f" %(spe))

    print(tpredy.to_string(index=False))
    print(tcont)
    print("Overall accuracy = %0.3f" %(tacc))
    print("Sensitivity = %0.3f" %(tsens))
    print("Specificity = %0.3f" %(tspe))

    #file output
    out = open(fname, 'w')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(predy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Resubstitution)",file=out)
        print(cont,file=out)
        print("",file=out)
        print("Model Summary (Resubstitution)",file=out)
        print("Overall accuracy = %0.3f" %(acc),file=out)
        print("Sensitivity = %0.3f" %(sens),file=out)
        print("Specificity = %0.3f" %(spe),file=out)
        print("",file=out)
        print(tpredy.to_string(index=False),file=out)
        print("",file=out)
        print("Confusion Matrix (Test)",file=out)
        print(tcont,file=out)
        print("",file=out)
        print("Model Summary (Test)",file=out)
        print("Overall accuracy = %0.3f" %(tacc),file=out)
        print("Sensitivity = %0.3f" %(tsens),file=out)
        print("Specificity = %0.3f" %(tspe),file=out)
    out.close()

In [151]:
DT1lv()

Write your train data file name : pid.dat
Write your test data file name : pidtest.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 2
Select the data header format(1 = with header or 2 = no header): 2
Select column# where your y vlaues exist: 8
Write your output file name : hw7.dat
ID  Actual class  Resub pred
  1             2           2
  2             2           2
  3             2           2
  4             2           2
  5             2           2
  6             2           2
  7             2           2
  8             2           2
  9             2           2
 10             2           2
 11             2           2
 12             2           2
 13             2           2
 14             2           2
 15             2           2
 16             2           2
 17             2           2
 18             2           1
 19             2           2
 20             2           2
 21             2           2
 22             2           2
 23           